In [1]:
import pandas as pd
from sklearn import *
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv("/data/credit-default.csv")
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,...,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job
0,< 0 DM,6,critical,radio/tv,1169,unknown,> 7 yrs,4,single male,none,...,real estate,67,none,own,2,1,1,yes,yes,skilled employee
1,1 - 200 DM,48,repaid,radio/tv,5951,< 100 DM,1 - 4 yrs,2,female,none,...,real estate,22,none,own,1,2,1,none,yes,skilled employee
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 yrs,2,single male,none,...,real estate,49,none,own,1,1,2,none,yes,unskilled resident
3,< 0 DM,42,repaid,furniture,7882,< 100 DM,4 - 7 yrs,2,single male,guarantor,...,building society savings,45,none,for free,1,1,2,none,yes,skilled employee
4,< 0 DM,24,delayed,car (new),4870,< 100 DM,1 - 4 yrs,3,single male,none,...,unknown/none,53,none,for free,2,2,2,none,yes,skilled employee


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
checking_balance        1000 non-null object
months_loan_duration    1000 non-null int64
credit_history          1000 non-null object
purpose                 1000 non-null object
amount                  1000 non-null int64
savings_balance         1000 non-null object
employment_length       1000 non-null object
installment_rate        1000 non-null int64
personal_status         1000 non-null object
other_debtors           1000 non-null object
residence_history       1000 non-null int64
property                1000 non-null object
age                     1000 non-null int64
installment_plan        1000 non-null object
housing                 1000 non-null object
existing_credits        1000 non-null int64
default                 1000 non-null int64
dependents              1000 non-null int64
telephone               1000 non-null object
foreign_worker          1000 non-null object
jo

In [5]:
y.value_counts()

1    700
2    300
Name: default, dtype: int64

In [16]:
target = "default"
X = df.drop(columns=target)

label_encoder = preprocessing.LabelEncoder()

y = label_encoder.fit_transform(df[target])

X = pd.get_dummies(X, drop_first=True)
features = X.columns

X_train, X_test, y_train, y_test = model_selection.train_test_split(X.values, y
                                            , test_size = 0.3, random_state = 1)

pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    #("scaler", preprocessing.StandardScaler()),
    ("est", tree.DecisionTreeClassifier(max_depth =3, random_state = 1
                                        , min_impurity_decrease = 0.001, min_samples_leaf = 10))
])

pipe.fit(X_train, y_train)

# Class prediction
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

# Probability estimate
y_train_prob = pipe.predict_proba(X_train)[:, 1]
y_test_prob = pipe.predict_proba(X_test)[:, 1]

print("Confusion matrix: \n", metrics.confusion_matrix(y_test, y_test_pred))



print("Training accuracy:", metrics.accuracy_score(y_train, y_train_pred))
print("Test accuracy:", metrics.accuracy_score(y_test, y_test_pred))

print("Training precision:", metrics.precision_score(y_train, y_train_pred))
print("Test precision:", metrics.precision_score(y_test, y_test_pred))

print("Training recall:", metrics.recall_score(y_train, y_train_pred))
print("Test recall:", metrics.recall_score(y_test, y_test_pred))

Confusion matrix: 
 [[193  21]
 [ 57  29]]
Training accuracy: 0.7471428571428571
Test accuracy: 0.74
Training precision: 0.6412213740458015
Test precision: 0.58
Training recall: 0.3925233644859813
Test recall: 0.3372093023255814


In [13]:
est = pipe.steps[-1][-1]

In [22]:
from sklearn.tree import export_graphviz
export_graphviz(est, out_file = "tree.dot", feature_names = X.columns, filled=True)
!dot -Tpng tree.dot -o tree.png


In [17]:
scores = model_selection.cross_val_score(pipe, X, y, cv = 5)
scores, np.mean(scores)

(array([0.75 , 0.695, 0.715, 0.695, 0.69 ]), 0.7089999999999999)

In [21]:
est = pipe.steps[-1][-1]
summary = pd.DataFrame({"importance": est.feature_importances_, "feature": features})
summary.sort_values("importance", ascending = False)

,importance,feature
9,0.533473,checking_balance_unknown
0,0.183929,months_loan_duration
24,0.086350,savings_balance_< 100 DM
11,0.085644,credit_history_fully repaid
39,0.060633,installment_plan_none
35,0.027046,other_debtors_none
27,0.022925,employment_length_1 - 4 yrs
36,0.000000,property_other
28,0.000000,employment_length_4 - 7 yrs
29,0.000000,employment_length_> 7 yrs


In [34]:
target = "default"
X = df.drop(columns=target)

label_encoder = preprocessing.LabelEncoder()

y = label_encoder.fit_transform(df[target])

X = pd.get_dummies(X, drop_first=True)
features = X.columns

X_train, X_test, y_train, y_test = model_selection.train_test_split(X.values, y
                                            , test_size = 0.3, random_state = 1)

pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    #("scaler", preprocessing.StandardScaler()),
    ("est", ensemble.RandomForestClassifier(n_estimators = 50, max_features= 0.5,  max_depth =10, random_state = 1
                                        , min_samples_leaf = 5))
])

pipe.fit(X_train, y_train)

# Class prediction
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

# Probability estimate
y_train_prob = pipe.predict_proba(X_train)[:, 1]
y_test_prob = pipe.predict_proba(X_test)[:, 1]

print("Confusion matrix: \n", metrics.confusion_matrix(y_test, y_test_pred))



print("Training accuracy:", metrics.accuracy_score(y_train, y_train_pred))
print("Test accuracy:", metrics.accuracy_score(y_test, y_test_pred))

print("Training precision:", metrics.precision_score(y_train, y_train_pred))
print("Test precision:", metrics.precision_score(y_test, y_test_pred))

print("Training recall:", metrics.recall_score(y_train, y_train_pred))
print("Test recall:", metrics.recall_score(y_test, y_test_pred))

Confusion matrix: 
 [[193  21]
 [ 52  34]]
Training accuracy: 0.8957142857142857
Test accuracy: 0.7566666666666667
Training precision: 0.9433962264150944
Test precision: 0.6181818181818182
Training recall: 0.7009345794392523
Test recall: 0.3953488372093023


In [35]:
est = pipe.steps[-1][-1]
summary = pd.DataFrame({"importance": est.feature_importances_, "feature": features})
summary.sort_values("importance", ascending = False)

,importance,feature
1,0.176181,amount
9,0.123503,checking_balance_unknown
0,0.120579,months_loan_duration
4,0.090566,age
7,0.065790,checking_balance_< 0 DM
2,0.050901,installment_rate
3,0.036436,residence_history
14,0.026135,purpose_car (new)
39,0.024140,installment_plan_none
43,0.018639,telephone_yes


In [36]:
est.estimators_

[DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
             max_features=0.5, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=5, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False,
             random_state=1791095845, splitter='best'),
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
             max_features=0.5, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=5, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False,
             random_state=2135392491, splitter='best'),
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
             max_features=0.5, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=5, min_samples_split=2,
             min_weight_fraction_l